In [1]:
# Import libraries and utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
import tensorflow as tf

from src.utils import (load_all, aggregate_player_stats, get_latest_valuation,
                      merge_player_data, fillna_and_scale, encode_categorical,
                      scatter_actual_vs_pred, compute_age)

pd.set_option('display.max_columns', 100)

In [2]:
# ─── 1. Build & clean main_df ─────────────────────────────────────────────────
data = load_all()
# Example: print available tables
for name, df in data.items():
    print(f'{name}: {df.shape}')
    
stats      = aggregate_player_stats(data['appearances'])
latest_val = get_latest_valuation(data['player_valuations'])
main_df    = merge_player_data(data['players'], stats, latest_val)



appearances: (1706806, 13)
clubs: (439, 17)
club_games: (148052, 11)
competitions: (44, 11)
games: (74026, 23)
game_events: (1035043, 10)
game_lineups: (2191911, 10)
players: (32601, 23)
player_valuations: (496606, 5)
transfers: (79646, 10)


In [3]:
# Remove the duplicated market_value column, rename the one we need
main_df = main_df.drop(columns='market_value_in_eur_x') \
                 .rename(columns={'market_value_in_eur_y': 'market_value_in_eur'})

# Compute age (assign result back into main_df)
main_df = compute_age(main_df)

# Drop any row where our target is missing
main_df = main_df.dropna(subset=['market_value_in_eur'])



In [4]:

# ─── 2. Decide which raw columns to drop (IDs, free‐form text, dates we’ve already engineered) ──

drop_cols = [
    'player_id',
    'player_code',
    'first_name', 'last_name', 'name', 'player_name',
    'url', 'image_url',
    'date_of_birth',          # we used this to create 'age'
    'contract_expiration_date',
    'agent_name',
    'current_club_name',      # free‐form text
    'from_club_name', 'to_club_name',
    # …add any other pure‐text or uninterpretable strings here if needed…
]

# Everything except drop_cols + the target itself
all_cols    = main_df.columns.tolist()
to_exclude  = drop_cols + ['market_value_in_eur']
feature_cols = [c for c in all_cols if c not in to_exclude]





In [5]:

# ─── 3. Auto-detect numeric vs. categorical among feature_cols ─────────────────────

# 3a. Numeric columns (int/float/Int64)
num_cols = main_df[feature_cols].select_dtypes(include=['int64','float64','Int64']).columns.tolist()

# 3b. Categorical columns (object or category)
cat_cols = main_df[feature_cols].select_dtypes(include=['object','category']).columns.tolist()

print("Numeric columns that will be scaled:", num_cols)
print("Categorical columns that will be one‐hot’ed:", cat_cols)

Numeric columns that will be scaled: ['last_season', 'current_club_id', 'height_in_cm', 'highest_market_value_in_eur', 'n_games', 'total_yellow', 'total_red', 'age']
Categorical columns that will be one‐hot’ed: ['country_of_birth', 'city_of_birth', 'country_of_citizenship', 'sub_position', 'position', 'foot', 'current_club_domestic_competition_id']


In [6]:

# ─── 4. Fill/missing + scale numerics, then one‐hot encode categoricals ─────────────

# Fill missing and scale numeric features
X_num, scaler = fillna_and_scale(main_df, num_cols)

# One‐hot encode all categorical features
X_cat, encoder = encode_categorical(main_df, cat_cols)

# Combine into one matrix
import numpy as np

if X_cat.shape[1] > 0:
    X_full = np.hstack([X_num, X_cat.values])
else:
    X_full = X_num

# Target (no NaN, because we dropped them above)
y = main_df['market_value_in_eur']




In [7]:

# ─── 5. Fit GBM & select “above‐median” features ─────────────────────────────────────

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel

gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_full, y)

selector   = SelectFromModel(gbr, prefit=True, threshold='median')
X_selected = selector.transform(X_full)

print("Total features before selection:", X_full.shape[1])
print("Selected features (importance ≥ median):", X_selected.shape[1])


KeyboardInterrupt: 

In [ ]:

# ─── 6. (Optional) Print exactly which feature names survived ────────────────────────

# Build the full list of feature‐names in the same order that GBM saw them:
feature_names = num_cols + X_cat.columns.tolist()

# selector.get_support() is a boolean mask of length len(feature_names)
kept_mask          = selector.get_support()
kept_feature_names = [name for name, keep in zip(feature_names, kept_mask) if keep]

print("\nFeatures kept by SelectFromModel (≥ median importance):")
for feat in kept_feature_names:
    print("  └", feat)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stop])

y_pred = model.predict(X_test).ravel()
scatter_actual_vs_pred(y_test, y_pred)


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Cell 6: Make predictions, clip extremes, compute metrics

# 1. Predict on the test set (log-space)
y_pred_log = model.predict(X_test).flatten()

# 2. Clip log-predictions to avoid absurd expm1 outputs
max_real_value = 12e7  # €300 million cap
max_log = np.log1p(max_real_value)
y_pred_log_clipped = np.clip(y_pred_log, a_min=0, a_max=max_log)

# 3. Convert back to real-EUR scale
y_test_exp = np.expm1(y_test)
y_pred_exp = np.expm1(y_pred_log_clipped)

# 4. Compute MAE and R^2 on the real-EUR scale
mae_real = mean_absolute_error(y_test_exp, y_pred_exp)
r2_real = r2_score(y_test_exp, y_pred_exp)

# 5. Compute R^2 in log-space
r2_log = r2_score(y_test, y_pred_log)

print(f"Test MAE (EUR scale, clipped): €{mae_real:,.2f}")
print(f"Test R^2 (EUR scale, clipped): {r2_real:.3f}")
print(f"Test R^2 (log1p scale): {r2_log:.3f}")

In [ ]:
# Cell 7: Scatter plot of Actual vs. Predicted (real-EUR), filtering out extreme predictions

# Only plot points where predicted < €500M, so axes stay readable
limit = 3e8
mask_plot = y_pred_exp < limit

# Apply the same mask to actuals so both arrays align
x_vals = y_test_exp[mask_plot]
y_vals = y_pred_exp[mask_plot]

# Compute min and max for X and Y
x_min, x_max = x_vals.min(), x_vals.max()
y_min, y_max = y_vals.min(), y_vals.max()

# Stretch limits just a tiny bit so points on the border aren’t cut off
x_pad = (x_max / x_min) ** 0.05  
y_pad = (y_max / y_min) ** 0.05

plt.figure(figsize=(8, 6))
plt.scatter(x_vals, y_vals, alpha=0.3)

# Set log–log scale
plt.xscale('log')
plt.yscale('log')

# Set axis limits to exactly cover the data (with a tiny 1% padding)
plt.xlim(x_min / x_pad, x_max * x_pad)
plt.ylim(y_min / y_pad, y_max * y_pad)

# Draw the y = x reference line over that same range
# On a log–log plot, the diagonal line from (min, min) to (max, max) remains straight.
plt.plot(
    [x_min, x_max],
    [x_min, x_max],
    color='green',
    linewidth=1,
    linestyle='--',
    zorder=0
)

plt.xlabel("Actual Market Value (EUR)")
plt.ylabel("Predicted Market Value (EUR, clipped)")
plt.title("Actual vs. Predicted Market Value (axes fit to data)")

plt.tight_layout()
plt.show()

In [ ]:
# Example: try different layer sizes
results = []
for size in [16, 32, 64, 128]:
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    y_pred = model.predict(X_test).flatten()
    y_pred_exp = np.expm1(y_pred)
    mae = mean_absolute_error(y_test_exp, y_pred_exp)
    results.append((size, mae))

print("Layer size vs MAE:")
for size, mae in results:
    print(f"  {size} units: MAE = €{mae:,.2f}")
